In [88]:
from src.eda.soporte_eda import *
import pandas as pd

In [73]:
conectar_bd()

(<connection object at 0x0000015CC2CF4E10; dsn: 'user=postgres password=xxx dbname=Proyecto_Sephora host=localhost port=5432', closed: 0>,
 <cursor object at 0x0000015CC2CEA880; closed: 0>)

Se parte de una query que trae los aspectos más relevantes de los productos, es decir, lo que les identifica (la url), el nombre, su descripción, la marca, la categoría y usbcategoría a la que pertenecen, el éxito que tienen (a partir del numero de valoraciones y la valoración) y el numero de variaciones de producto. Otras características consideradas más secundarias y menos relevantes sobre los productos son el acabado, la textura, la responsabilidad etc. Características que se emplearán en el análisis más específico para poder concretar mejor en el producto final recomendado a lanzar por Sephora Collection, pero que en este primer análisis general no se tomarán en cuenta, pues este análisis simplemente tiene en cuenta poder obtener una primera aproximación sobre la tienda de Sephora y sobretodo, poder comenzar a posicionar a Sephora Collection en comparación con el resto de las marcas. 

In [76]:
query_general = """
select 
    p.url_producto,
    p.nombre,
    p.descripcion,
    m.nombre_marca, 
    c.nombre_categoria,
    s.nombre_subcategoria,
    h.precio,
    h.numero_valoraciones,
    h.valoracion,
    h.numero_variaciones
from productos p 
join marcas m on p.id_marca = m.id_marca
left join categorias c on p.id_categoria = c.id_categoria
left join subcategorias s on p.id_subcategoria = s.id_subcategoria
join historico h on p.id_producto = h.id_producto;
"""

In [77]:
df = ejecutar_query(query_general)
df.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,https://www.sephora.es/p/soft-lit-naturally-lu...,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50
1,https://www.sephora.es/p/triclone-skin-tech-fo...,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51
2,https://www.sephora.es/p/triclone-skin-tech-hy...,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30
3,https://www.sephora.es/p/color-fuse-talc-free-...,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8,6
4,https://www.sephora.es/p/bio-radiant-gel-powde...,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8,5


In [78]:
df.shape

(1829, 10)

De primeras se cuenta con las 10 columnas ya previamente mencionadas y 1829 filas, es decir, 1829 productos de maquillaje en todo Sephora.

In [79]:
def info_reporte(dataframe):
    """
    Genera un informe sobre los valores nulos y tipos de datos de un DataFrame.

    Args:
        dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
        pd.DataFrame: Un DataFrame con el número y porcentaje de valores nulos por columna,
                      junto con el tipo de dato de cada columna.
    """
    df_report = pd.DataFrame()
    df_report["Numero_nulos"] = dataframe.isnull().sum()
    df_report["Porcentaje_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0]*100), 2)
    df_report["Tipo_dato"] = dataframe.dtypes
    return df_report

In [80]:
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
nombre,3,0.16,object
descripcion,98,5.36,object
nombre_marca,0,0.00,object
nombre_categoria,9,0.49,object
nombre_subcategoria,33,1.80,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64
numero_variaciones,0,0.00,int64


Con respecto a los valores nulos, encontramos 3 productos con nulos en el nombre, 98 con nulos en la descripción, 9 con nulos en la categoría y 33 con nulos en la subcategoría. Estos nulos no son necesarios tratarlos ya que son nulos que tienen que estar ahí, pues correctamente hay determinados productos de los cuales se desconoce su nombre, su descripción, su categoría o su subcategoría. De todas formas, el % de nulos que hay en cada una de estas columnas es mínimo por lo que se van a mantener estos productos ya que el resto de información puede resultar relevante para el análisis. Por otro lado, en cuanto al tipo de dato, en la extracción hay un proceso de transformación asegurando que cada columna se extraiga con el tipo de dato correcto y por lo tanto, todas las columnas tienen el tipo de dato correcto.

In [81]:
df["url_producto"].duplicated().sum()

np.int64(7)

In [84]:
df.drop_duplicates(subset=["url_producto"], inplace=True)
df.shape

(1822, 10)

En cuanto a los duplicados, no debería haber ningún duplicado por el campo de url del producto, ya que este campo es el identificador único de los productos. No debería haber ningún duplicado porque al haber realizado únicamente un scrapeo (el primero), los productos no deben aparecer más de una vez en la tabla de histórico. Por ahora no se ha logrado identificar en el código donde puede estar el punto donde se comete este error generando duplicados sobre 7 productos (identifica como que los productos ya están en la tabla "productos" y por lo tanto inserta una nueva fila a la tabla "historico", convirtiéndose en un duplicado erróneo). De momento, y al ser muy pocos los duplicados, para poder realizar un análisis con los datos correctamente recogidos, elimino directamente las filas duplicadas y por lo tanto nos quedamos con el número correcto de productos de maquillaje en toda Sephora, que son 1822. 

### Número total de productos en Sephora

In [85]:
numero_total_productos = df.shape[0]
print(f"Numero total de productos: {numero_total_productos}")

Numero total de productos: 1822


In [86]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,1822.0,37.201850,18.895532,3.99,25.0,34.99,45.00,169.0
numero_valoraciones,1822.0,590.726674,1707.459476,0.00,3.0,42.50,406.75,20035.0
valoracion,1822.0,3.799671,1.543679,0.00,3.9,4.40,4.70,5.0
numero_variaciones,1822.0,4.249177,7.088864,0.00,0.0,2.00,6.00,60.0


Descriptivos de las variables numéricas: 
- Precio: El precio de los productos en Sephora oscila entre 3,99 € y 169 €. La media se sitúa en 37,20 €, mientras que la mediana es de 34,99 €, lo que indica que la distribución está sesgada a la derecha. Esto significa que hay una mayoría de productos con precios moderados, pero algunos artículos significativamente más caros elevan el promedio. El 25% de los productos cuesta menos de 25 €, el 50% menos de 34,99 €, y el 75% menos de 45 €. Solo el 25% de los productos supera los 45 €, y un pequeño número alcanza precios muy altos, lo que sugiere la existencia de productos premium que pueden considerarse outliers dentro de la gama de precios.
- Número de valoraciones: La cantidad de valoraciones por producto varía de forma muy amplia, desde 0 hasta 20.035. La media es de 590,73 valoraciones, pero la mediana es de solo 42,5, lo que refleja una distribución altamente sesgada a la derecha. Esto indica que la mayoría de los productos tiene muy pocas valoraciones, mientras que unos pocos concentran una gran cantidad, elevando considerablemente el promedio. Aproximadamente el 25% de los productos cuenta con menos de 3 valoraciones, la mitad tiene menos de 42,5, y el 75% no supera las 406,75 valoraciones. Solo una cuarta parte de los productos supera ese umbral, y algunos alcanzan cifras muy altas, evidenciando casos claramente atípicos. Esta distribución sugiere que la visibilidad o la popularidad está muy concentrada en unos pocos productos dentro del catálogo.
- Valoración: Las valoraciones de los productos se mueven entre 0 y 5 puntos. La media es de 3,80, mientras que la mediana se sitúa en 4,40, lo que revela una ligera asimetría hacia la izquierda. Esto sugiere que la mayoría de los productos recibe buenas puntuaciones, pero algunos obtienen valoraciones muy bajas, lo que reduce el promedio. Un cuarto de los productos tiene una valoración inferior a 3,9, mientras que el 50% no supera los 4,4 y el 75% se sitúa por debajo de 4,7. Solo una pequeña proporción logra valoraciones superiores a ese nivel, alcanzando incluso el máximo de 5. Por lo tanto, aunque las valoraciones son en general altas, existen productos con puntuaciones bajas que podrían indicar problemas de calidad o satisfacción del cliente, y que pueden considerarse outliers negativos.
- El número de variaciones disponibles por producto, como diferentes tonos o tamaños, varía desde 0 hasta 60. La media es de 4,25 variaciones por producto, mientras que la mediana es de 2, lo cual indica una distribución fuertemente sesgada a la derecha. Esto significa que la mayoría de los productos tiene pocas opciones, pero algunos ofrecen una amplia gama de variaciones, lo que eleva la media general. El 25% de los productos no tiene ninguna variación, la mitad tiene dos o menos, y el 75% no supera las seis variaciones. Solo una minoría supera este número, y unos pocos productos llegan a tener hasta 60, situándose claramente como outliers. Esta situación es habitual en productos como bases de maquillaje, pintalabios o correctores, donde se requiere una gran diversidad de tonos para adaptarse a diferentes perfiles de consumidores.

### Valores representativos de las variables numéricas

In [109]:
mediana_precio = df["precio"].median()
print(f"Mediana precio: {mediana_precio}")
mediana_num_valoraciones = df["numero_valoraciones"].median()
print(f"Mediana numero valoraciones: {mediana_num_valoraciones}")
mediana_valoracion = df["valoracion"].median()
print(f"Mediana valoracion: {mediana_valoracion}")
mediana_num_variaciones = df["numero_variaciones"].median()
print(f"Mediana numero variaciones: {mediana_num_variaciones}")

Mediana precio: 34.99
Mediana numero valoraciones: 42.5
Mediana valoracion: 4.4
Mediana numero variaciones: 2.0


### Observación de outliers de variables numéricas

In [117]:
def detectar_outliers(df, columna, modo='superior'):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    umbral_inferior = Q1 - 1.5 * IQR
    umbral_superior = Q3 + 1.5 * IQR

    if modo == 'superior':
        outliers = df[df[columna] > umbral_superior]
    else:
        outliers = df[df[columna] < umbral_inferior]

    return outliers[['nombre', 'descripcion', 'nombre_marca', 'nombre_categoria', 'nombre_subcategoria', 'precio', 'numero_valoraciones', 'valoracion', 'numero_variaciones']]

outliers_precio = detectar_outliers(df, 'precio', modo='superior')
outliers_valoraciones = detectar_outliers(df, 'numero_valoraciones', modo='superior')
outliers_valoracion = detectar_outliers(df, 'valoracion', modo='inferior')
outliers_variaciones = detectar_outliers(df, 'numero_variaciones', modo='superior')

In [118]:
outliers_precio.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
12,HD Skin All-In-One Palette,Paleta de maquillaje todo en uno,Make Up For Ever,Paletas maquillaje,Paleta rostro,85.0,263,4.0,2
15,Diorshow 5 Couleurs edición limitada cannage,Paleta de sombras de ojos,Dior,Ojos,Paleta de sombras de ojos,76.0,0,0.0,4
22,Invisible UV Flawless Poreless Primer,Base de maquillaje,Charlotte Tilbury,Rostro,Primer & spray fijador,79.0,3,4.7,0
101,Green Flash,Kit pro Hortencia & Poppy Red,Manucurist,Estuches maquillaje,Estuches de manicura,86.0,0,0.0,0
189,HD Skin Sculting Palette,Paleta Modeladora del rostro Crema,Make Up For Ever,Paletas maquillaje,Paleta rostro,84.0,179,4.6,0


In [119]:
outliers_precio.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,56.0,103.462321,23.587274,76.0,85.0,96.00,115.2475,169.0
numero_valoraciones,56.0,101.267857,282.616701,0.0,2.0,16.50,44.2500,1959.0
valoracion,56.0,3.800000,1.824978,0.0,4.0,4.65,4.9000,5.0
numero_variaciones,56.0,3.196429,5.955653,0.0,0.0,0.00,4.0000,30.0


In [120]:
outliers_precio.describe(include='object').T

,count,unique,top,freq
nombre,56,54,Green Flash,2
descripcion,51,39,Paleta de sombras de ojos,7
nombre_marca,56,15,Sisley,19
nombre_categoria,55,7,Rostro,27
nombre_subcategoria,55,16,Base de maquillaje,12


Los productos outliers por precio en Sephora tienen un precio significativamente más alto, con una mediana de 96.00, en comparación con los productos generales, cuya mediana es de 34.99. Además, estos productos también tienen mejores valoraciones, con una mediana de 4.65, superior a la mediana general de 4.40. En cuanto al número de valoraciones, los outliers tienen una mediana de 16.50, menor a la mediana de 42.50 de los productos generales. En términos de variaciones, la mediana de los outliers es de 0.00, indicando que estos productos tienen menos opciones disponibles en comparación con los productos generales, cuya mediana de variaciones es de 2.00. Estos productos provienen principalmente de marcas de lujo como Sisley y se concentran en las categorías de Rostro y Base de maquillaje. En resumen, los productos más caros de Sephora tienden a tener mejores valoraciones, menos valoraciones y menos opciones disponibles en comparación con los productos más económicos.

In [121]:
outliers_valoraciones.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
34,Vanish™ Airbrush Concealer,Corrector antiojeras,Hourglass,Rostro,Antiojeras & corrector,42.00,3247,4.3,29
43,Pout Preserve Lip Treatment,Cuidado labial hidratante y antiedad,Olehenriksen,Labios,Bálsamo labial,21.00,1185,4.4,8
54,Limtless Lash Mascara,Mascara,Ilia,Ojos,Máscara de pestañas,35.00,4117,4.5,2
76,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...",Valentino,Labios,Gloss/ Brillo labial,39.99,1020,4.7,6
89,Easy Bake Setting Spray,Spray fijador de maquillaje 16 horas,Huda Beauty,Rostro,Primer & spray fijador,35.00,1014,4.6,2


In [122]:
outliers_valoraciones.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,256.0,36.228828,12.815420,9.99,27.7425,35.0,42.0,79.0
numero_valoraciones,256.0,3380.621094,3378.514686,1014.00,1397.5000,2292.5,3711.0,20035.0
valoracion,256.0,4.321094,0.307591,2.90,4.2000,4.4,4.6,4.8
numero_variaciones,256.0,4.949219,7.976028,0.00,0.0000,2.0,7.0,49.0


In [123]:
outliers_valoraciones.describe(include='object').T

,count,unique,top,freq
nombre,256,246,Spike Valentino,2
descripcion,251,210,Máscara,6
nombre_marca,256,36,Benefit Cosmetics,45
nombre_categoria,256,8,Rostro,109
nombre_subcategoria,254,32,Máscara de pestañas,43


Los productos con un número de valoraciones excepcionalmente alto (outliers) tienen una mediana de 2,292.5 valoraciones, muy por encima de la mediana general de 42.5, lo que indica una altísima popularidad. No se corresponden con ser productos más baratos (mediana de 35.00, casi igual a los 34.99 generales), y presentan una valoración igual a los productos generales (mediana de 4,4), lo que sugiere una percepción positiva mantenida incluso con un volumen masivo de opiniones. En cuanto a las variaciones, su mediana es de 2.0, igual que la de los productos generales, lo que indica una disponibilidad de opciones promedio. A nivel de marcas, destacan nombres populares como Benefit Cosmetics, y en categorías, predominan productos de Rostro, especialmente la Máscara de pestañas, que aparece como la subcategoría más frecuente. En conjunto, estos productos mantienen un precio promedio en la tienda y son muy bien reconocidos, manteniendo buenas valoraciones a gran escala.

In [124]:
outliers_valoracion.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
7,Cherry Mouth,Estuche Trío de Labios,Fenty Beauty,Estuches maquillaje,Estuches de labiales,51.0,0,0.0,0
9,Lip Injection Plumping Icons,Estuche Labios,Too Faced,Estuches maquillaje,Estuches de labiales,50.0,0,0.0,0
14,Master Mascara™,Máscara voluminizadora,Makeup By Mario,Ojos,Máscara de pestañas,33.0,0,0.0,0
15,Diorshow 5 Couleurs edición limitada cannage,Paleta de sombras de ojos,Dior,Ojos,Paleta de sombras de ojos,76.0,0,0.0,4
16,Even Better Light Reflecting Primer,Prebase iluminadora,Clinique,Rostro,Primer & spray fijador,40.0,0,0.0,0


In [125]:
outliers_valoracion.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,259.0,38.490347,19.442039,7.99,27.99,36.0,46.0,144.0
numero_valoraciones,259.0,0.440154,2.122072,0.00,0.00,0.0,0.0,22.0
valoracion,259.0,0.186873,0.580615,0.00,0.00,0.0,0.0,2.6
numero_variaciones,259.0,2.637066,4.331194,0.00,0.00,0.0,4.0,26.0


In [126]:
outliers_valoracion.describe(include='object').T

,count,unique,top,freq
nombre,256,243,Moving Lights,6
descripcion,233,206,Lápiz de labios,3
nombre_marca,259,53,Sephora Collection,24
nombre_categoria,255,10,Rostro,66
nombre_subcategoria,251,43,Barra de labios,23


Los productos con valoraciones excepcionalmente bajas presentan una mediana de 0.0, muy por debajo de la mediana general de 4.4, lo que indica una percepción claramente negativa o problemas de calidad o experiencia. Además, estos productos prácticamente no tienen valoraciones (mediana también de 0), lo que refuerza la baja visibilidad o el desinterés por parte de los usuarios. A nivel de precio, su mediana (36.0) es similar a la del conjunto general (34.99), por lo que no son ni especialmente baratos ni caros. En cuanto a variaciones, la mediana es de 0, también por debajo del global (2.0), lo que sugiere menor diversidad de opciones o formatos. Predominan marcas como Sephora Collection, y las categorías más frecuentes son productos de Rostro, destacando subcategorías como Barra de labios o Lápiz de labios. En conjunto, se trata de productos con escasa interacción y muy baja valoración, lo que puede indicar desinterés, problemas de lanzamiento o experiencias de usuario deficientes.

In [127]:
outliers_variaciones.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50
1,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51
2,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30
13,Dior Addict,Barra de labios brillante 90 % de ingredientes...,Dior,Labios,Barra de labios,51.00,977,4.5,26
34,Vanish™ Airbrush Concealer,Corrector antiojeras,Hourglass,Rostro,Antiojeras & corrector,42.00,3247,4.3,29


In [128]:
outliers_variaciones.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,114.0,41.629386,16.368728,4.99,32.0,41.49,51.0,99.0
numero_valoraciones,114.0,1048.956140,2804.771208,0.00,20.0,69.50,585.5,17079.0
valoracion,114.0,4.103509,1.064880,0.00,4.1,4.30,4.6,5.0
numero_variaciones,114.0,25.885965,9.463258,16.00,19.0,23.00,30.0,60.0


In [129]:
outliers_variaciones.describe(include='object').T

,count,unique,top,freq
nombre,114,110,Rouge Is Not My Name,2
descripcion,112,81,Corrector,12
nombre_marca,114,34,Sephora Collection,13
nombre_categoria,114,4,Rostro,82
nombre_subcategoria,114,14,Base de maquillaje,51


Los productos con un número inusualmente alto de variaciones (mediana de 23 frente a 2 en el conjunto general) destacan por ofrecer una gran diversidad de tonos, formatos o versiones. A pesar de su complejidad de catálogo, mantienen una valoración sólida (mediana de 4.3, similar al total), lo que indica que la variedad no compromete la calidad percibida. Además, presentan un volumen notable de valoraciones (mediana de 69.5 frente a 42.5 en general), lo que sugiere buena aceptación por parte de los usuarios. Su precio (mediana de 41.49) es algo superior al promedio del catálogo, posiblemente asociado a la amplitud de oferta. Estos productos se concentran sobre todo en la categoría de Rostro, especialmente en la subcategoría de Base de maquillaje, y destacan marcas como Sephora Collection. En conjunto, se trata de productos versátiles y bien valorados, con alta interacción del cliente.

In [87]:
df.describe(include="object").T

,count,unique,top,freq
url_producto,1822,1822,https://www.sephora.es/p-640056.html,1
nombre,1819,1646,Moving Lights,14
descripcion,1724,1204,Paleta de sombras de ojos,44
nombre_marca,1822,79,Sephora Collection,198
nombre_categoria,1813,13,Rostro,627
nombre_subcategoria,1789,48,Máscara de pestañas,148


### Valores representativos de las variables categóricas

In [130]:
numero_marcas = df["nombre_marca"].nunique()
print(f"Sephora cuenta con {numero_marcas} marcas diferentes")
numero_categorias = df["nombre_categoria"].nunique()
print(f"Sephora cuenta con {numero_categorias} categorias diferentes")
numero_subcategorias = df["nombre_subcategoria"].nunique()
print(f"Sephora cuenta con {numero_subcategorias} subcategorias diferentes")

Sephora cuenta con 79 marcas diferentes
Sephora cuenta con 13 categorias diferentes
Sephora cuenta con 48 subcategorias diferentes


### Recuento de productos por marca

In [143]:
df_marca_count = df.groupby('nombre_marca').size().reset_index(name='count')
df_marca_count['%/Total'] = ((df_marca_count['count'] / numero_total_productos) * 100).round(2)
df_marca_count = df_marca_count.sort_values(by='%/Total', ascending=False)
df_marca_count.head(5)

,nombre_marca,count,%/Total
63,Sephora Collection,198,10.87
5,Benefit Cosmetics,86,4.72
10,Charlotte Tilbury,77,4.23
42,Make Up For Ever,66,3.62
15,Dior,65,3.57


Sephora Collection es la marca que más presencia tiene en número de productos ofertados en Sephora (cuenta con el 10,87% de los productos), seguida de Benefit Cosmetics y Charlotte Tilbury (4,72% y 4,23% respectivamente). Por el lado contrario, marcas como The Ordinary, Urban Decay, YEPODA y otras más, cuentan con tan solo el 0,05% de productos ofertados en Sephora.

### Recuento de productos por categoria 

In [145]:
df_cat_count = df.groupby('nombre_categoria').size().reset_index(name='count')
df_cat_count['%/Total'] = ((df_cat_count['count'] / numero_total_productos) * 100).round(2)
df_cat_count = df_cat_count.sort_values(by='%/Total', ascending=False)
df_cat_count.head(5)

,nombre_categoria,count,%/Total
10,Rostro,627,34.41
4,Ojos,372,20.42
3,Labios,343,18.83
8,Pinceles & accesorios,167,9.17
0,Cejas,100,5.49


La categoría con más presencia en Sephora es el Rostro con un 34,41% de productos ofertados en la tienda. Le siguen los Ojos con un 20,42% y los Labios con un 18,83%. Por el lado contrario, las categorías con menos presencia serían los Sacapuntas y Pestañas postizas con un 0,22%, los Rizadores de pestañas con un 0,16% y los Papeles matificantes con un 0,05%.

### Recuento de procuctos por subcategoria 

In [147]:
df_subcat_count = df.groupby('nombre_subcategoria').size().reset_index(name='count')
df_subcat_count['%/Total'] = ((df_subcat_count['count'] / numero_total_productos) * 100).round(2)
df_subcat_count = df_subcat_count.sort_values(by='%/Total', ascending=False)
df_subcat_count.head(5)

,nombre_subcategoria,count,%/Total
32,Máscara de pestañas,148,8.12
2,Barra de labios,126,6.92
8,Colorete,113,6.20
4,Base de maquillaje,107,5.87
42,Primer & spray fijador,104,5.71


Las subcategorías con más presencia dentro de Sephora son las Máscaras de pestañas (8,12%), las Barras de labios (6,92%) y los Coloretes (6,20%). Por el lado contrario, las que tienen menos presencia serían los Limpiadores de pinceles y las Paletas multi-usos con un 0,05% cada una.

Productos considerados ya bastante exitosos dentro de Sephora (me baso sobretodo en las medianas aunque redondeando un poco los valores al alza, es decir, aproximándolo incluso un poco más por encima del 50% de la distribución. decido marcar los limites en valoracion de 4,5 que ya considero que es bastante buena valoracion y ademas el percentil 75 esta en 4,7, es decir que no se aleja casi de la valoracion que tan solo logran tener mas del 25% de los productos. en el caso del numero de valoraciones basandome tambien un poco en la mediana aunque redondeando al alza he fijado el limite en 50 valoraciones pues considero que conocer que tan solo el 50% de los productos tiene mas de 42 valoraciones ya es indicador de que de por si aquellos productos que tienen mas valoraciones tampoco tienen demasiadas con excepcion de aquellos productos "outliers" que si cuentan con muchas valoraciones. y he escogido la mediana tan bien mas bien que el percentil 75 porque al final aqui estoy combinando ambos aspectos, el numero de valoraciones y la valoracion por lo que considero que ya de por si es bastante buen indicador que combine lo que por separado tan solo consiguen menos del 50% de los productos)

In [57]:
exitosos = df[(df["valoracion"] >= 4.5) & (df["numero_valoraciones"] >= 50)]

print("Número de productos exitosos:", len(exitosos))

Número de productos exitosos: 393


aqui indicar quizas que % suponen sobre el total de productos de sephora 

Productos exitosos por marca 

In [53]:
exitosos_por_marca = exitosos.groupby("nombre_marca")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
exitosos_por_marca.head(10)

,nombre_marca,url_producto
3,Benefit Cosmetics,35
37,Sephora Collection,32
30,Natasha Denona,29
0,Anastasia Beverly Hills,27
19,Huda Beauty,23
25,Make Up For Ever,21
23,Lancôme,18
29,Nars,18
18,Hourglass,14
5,Charlotte Tilbury,12


Productos exitosos por categorias

In [54]:
exitosos_por_categoria = exitosos.groupby("nombre_categoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
exitosos_por_categoria.head(10)

,nombre_categoria,url_producto
7,Rostro,152
3,Labios,69
4,Ojos,63
6,Pinceles & accesorios,46
0,Cejas,26
5,Paletas maquillaje,20
1,Desmaquillante,10
9,Uñas,5
2,Estuches maquillaje,1
8,Sacapuntas,1


Productos exitosos por subcategorias

In [55]:
exitosos_por_subcategoria = exitosos.groupby("nombre_subcategoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
exitosos_por_subcategoria.head(10)

,nombre_subcategoria,url_producto
8,Colorete,43
6,Brocha rostro,30
25,Paleta de sombras de ojos,29
2,Barra de labios,28
33,Sombras de ojos,23
19,Iluminador,19
24,Máscara de pestañas,18
4,Base de maquillaje,16
28,Polvos bronceadores,15
21,Lápiz & polvo para cejas,14


en estas tablas de group by lo que puedo hacer es añadir una columna que sea % sobre el total (en las de arriba sobre el total de productos general de sephora y en estas de productos exitosos quizas mejor sobre el total de productos que tienen exito en sephora)

observar si estos productos mas exitosos estan correspondidos con precios mas altos o mas bajos y puede haber ahi cierta relacion

In [60]:
precio_medio_exitosos = exitosos["precio"].mean()
precio_medio_exitosos

np.float64(38.9875572519084)

Al ser el precio medio casi el mismo que de los productos de Sephora en general (Incluso el de los exitosos un pelin mas elevado), sugiere que precios mas bajos no son lo que justifican principalmente a los productos mas exitosos. influiran otros factores

precios medios por marcas 

In [62]:
precio_medio_marca = df.groupby("nombre_marca")["precio"].mean().reset_index().sort_values("precio", ascending=False)
precio_medio_marca.tail(10)

,nombre_marca,precio
9,Caudalie,19.666667
3,BEAUTY OF JOSEON,18.000000
26,Glow Recipe,17.500000
7,Byoma,16.500000
41,Lanolips,15.937500
63,Sephora Collection,15.846061
77,Youth To The People,15.000000
70,The Ordinary,14.990000
61,SVR,13.333333
46,Merci Handy,12.990000


precios medios por categorias

In [63]:
precio_medio_categoria = df.groupby("nombre_categoria")["precio"].mean().reset_index().sort_values("precio", ascending=False)
precio_medio_categoria.head(10)

,nombre_categoria,precio
5,Paletas maquillaje,61.396400
10,Rostro,42.507528
1,Desmaquillante,38.136552
2,Estuches maquillaje,36.521905
4,Ojos,35.445457
8,Pinceles & accesorios,34.103750
3,Labios,32.021134
0,Cejas,31.107400
7,Pestañas postizas,22.250000
11,Sacapuntas,20.745000


precios medios por subcategorias

In [64]:
precio_medio_subcategoria = df.groupby("nombre_subcategoria")["precio"].mean().reset_index().sort_values("precio", ascending=False)
precio_medio_subcategoria.head(10)

,nombre_subcategoria,precio
36,Paleta rostro,59.664000
34,Paleta de sombras de ojos,59.517333
33,Paleta de contouring,55.000000
4,Base de maquillaje,54.913645
35,Paleta multi-usos,51.000000
40,Polvos matificantes,46.499063
18,Estuches de manicura,45.396000
10,Crema de color,44.097000
39,Polvos bronceadores,44.017885
12,Desmaquillante rostro,43.256000


In [67]:
precio_medio_marca_subcat = (
    df.groupby(["nombre_categoria", "nombre_marca"])["precio"]
    .mean()
    .reset_index()
    .sort_values(by=["nombre_categoria", "precio"], ascending=[True, False])
)
precio_medio_marca_subcat

,nombre_categoria,nombre_marca,precio
24,Cejas,Sisley,60.000000
9,Cejas,Grande Cosmetics,54.990000
27,Cejas,Valentino,43.000000
11,Cejas,Hourglass,37.000000
5,Cejas,Dior,35.666667
...,...,...,...
274,Sacapuntas,Sephora Collection,3.990000
275,Uñas,Dior,35.666667
276,Uñas,Isdin,27.000000
277,Uñas,Manucurist,17.707143


esto tengo que darle una vuelta para ver como puedo analizarlo porque aqui con todas es imposible (en el dashboard si que podria) y sino aqui escoger alguna concreta

ver tambien si meto algo aqui de numero de variaciones

marcas que dominan cada categoria de productos

In [69]:
# Contar el número de productos por marca dentro de cada categoría
marcas_por_categoria = df.groupby(["nombre_categoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")

# Encontrar la marca con más productos para cada categoría
marcas_dominantes_por_categoria = marcas_por_categoria.loc[marcas_por_categoria.groupby("nombre_categoria")["numero_productos"].idxmax()]

# Mostrar los resultados
marcas_dominantes_por_categoria

,nombre_categoria,nombre_marca,numero_productos
0,Cejas,Anastasia Beverly Hills,21
35,Desmaquillante,Collistar,5
77,Estuches maquillaje,Sephora Collection,9
127,Labios,Sephora Collection,24
168,Ojos,Sephora Collection,53
188,Paletas maquillaje,Natasha Denona,10
193,Papeles matificantes,Rem Beauty,1
194,Pestañas postizas,Huda Beauty,4
216,Pinceles & accesorios,Sephora Collection,48
219,Rizador pestañas,Dior Backstage,1


marcas que dominan cada subcategoria de productos

In [70]:
# Contar el número de productos por marca dentro de cada categoría
marcas_por_subcategoria = df.groupby(["nombre_subcategoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")

# Encontrar la marca con más productos para cada categoría
marcas_dominantes_por_subcategoria = marcas_por_subcategoria.loc[marcas_por_subcategoria.groupby("nombre_subcategoria")["numero_productos"].idxmax()]

# Mostrar los resultados
marcas_dominantes_por_subcategoria

,nombre_subcategoria,nombre_marca,numero_productos
0,Aceite desmaquillante,CHANEL,2
47,Antiojeras & corrector,Tarte,9
54,Barra de labios,Charlotte Tilbury,11
83,Barra de labios líquida,Benefit Cosmetics,3
107,Base de maquillaje,Fenty Beauty,7
142,Base de sombra de ojos,Too Faced,3
161,Brocha rostro,Sephora Collection,25
179,Bálsamo labial,Lanolips,13
198,Colorete,Benefit Cosmetics,20
241,Contouring,Sephora Collection,3


## Resumen Conclusiones Más Relevantes Análisis General Sephora 
- Cuenta con 1822 productos de maquillaje.
- Cuenta con 79 marcas, 13 categorías y 48 subcategorías.
- Valores representativos
    - Precio: 34,99
    - Número de valoraciones: 42,5
    - Valoración: 4,4
    - Número de variaciones: 2
- No se observa una relación directa entre el precio y el reconocimiento de los productos. Los productos más caros tienen buenas valoraciones, aunque un número menor de valoraciones. Los productos con mayor número de valoraciones y buenas reseñas no son necesariamente más baratos, sino que tienen un precio promedio. Por otro lado, los productos con menor éxito y visibilidad tampoco están relacionados con precios más altos, sino que también mantienen un precio promedio.
- Sephora Collection tiene buena presencia (número de productos ofertados): 10,87%
- El Rostro, los Ojos y los Labios son las categorías con más presencia en Sephora. En cuanto a subcategorías son las Máscaras de pestañas, las Barras de labios y los Coloretes